# Time Dependent Heat Transfer

This is the implementation for time dependent heat transfer in 2D using FEniCSx.
The boundary-value problem is described by:
$$
\begin{aligned}
\frac{\partial u}{\partial t} &= \Delta u(x) + f(x) &\quad x\in \Omega \times (0,T] \\
u(x) &= u_D(x) &\quad x\in \partial \Omega \times (0,T] \\
u(x) &= u_0 &\quad at \quad t = 0
\end{aligned}
$$

## Variational formulation

### Crank-Nicholson time stepper

The sampling of a PDE at time $t_{n+1}$ is given by:
$$\left(\frac{\partial u}{\partial t}\right)^{n+1} = \Delta u^{n+1} + f^{n+1}$$

The time-derivative can be approximated using Crank-Nicholsam time stepper as:
$$
\begin{aligned}
\frac{u^{n+1} - u^n}{\Delta t} = \frac{1}{2}\left(\Delta u^{n+1} + \Delta u^n + f^{n+1} + f^n\right)\\
u^{n+1} - \frac{\Delta t}{2}\Delta u^{n+1} = u^n + \frac{\Delta t}{2}\left(\Delta u^{n} + f^n + f^{n+1}\right)
\end{aligned}
$$

### Weak formulation

To calculate the variational formulation, we multiply the equation by a test function $v$ and integrate over $\Omega$:
$$\int_\Omega \left( u^{n+1} v - \frac{\Delta t}{2} \Delta u^{n+1} v \right) dx = \int_\Omega u^n v dx + \frac{\Delta t}{2} \int_\Omega \left( \Delta u^n v + f^{n+1} v + f^n v \right) dx$$

Using integration by parts to the reduce the higher order terms:
$$\int_\Omega u^{n+1} v dx + \frac{\Delta t}{2}\int_\Omega \nabla u^{n+1} \cdot \nabla v dx - \frac{\Delta t}{2}\int_{\partial\Omega} \frac{\partial u^{n+1}}{\partial n} v ds
    = \int_\Omega u^n v dx - \frac{\Delta t}{2}\int_\Omega \nabla u^n \cdot \nabla v dx + \frac{\Delta t}{2}\int_{\partial\Omega} \frac{\partial u^n}{\partial n} v ds + \frac{\Delta t}{2} \int_\Omega \left( f^{n+1} v + f^n v \right) dx$$
Since the integral term on the boundary is $0$, the equation becomes
$$\int_\Omega u^{n+1} v dx + \frac{\Delta t}{2}\int_\Omega \nabla u^{n+1} \cdot \nabla v dx = \int_\Omega u^n v dx - \frac{\Delta t}{2}\int_\Omega \nabla u^n \cdot \nabla v dx + \frac{\Delta t}{2} \int_\Omega \left( f^{n+1} v + f^n v \right) dx$$
leading to
$$
\begin{aligned}
a(u^{n+1},v) &:= \int_\Omega \left(u^{n+1} v + \frac{\Delta t}{2}\nabla u^{n+1} \cdot \nabla v \right)dx\\
L_{n+1}(v) &:= \int_\Omega \left(u^n v + \frac{\Delta t}{2}\left( f^{n+1} v + f^n v - \nabla u^n \cdot \nabla v \right) \right)dx
\end{aligned}
$$

## Problem definition

- Domain: $\Omega = [0,1] \times [0,1]$
- Dirichlet BCs: $u = 0$ on $\{(0,y) \cup (2,y)\} \in \partial\Omega$
- Forcing fn: $f = 0$

In [2]:
# Import Libraries
from mpi4py import MPI
from dolfinx import mesh, fem, plot, io
import numpy as np
import ufl
from dolfinx.fem.petsc import assemble_matrix, create_vector, assemble_vector, apply_lifting, set_bc
from petsc4py import PETSc
import pyvista
import matplotlib as mpl
from pathlib import Path

In [ ]:
# Define time parameters
t = 0.0 # Start time
T = 1.0 # End time
num_steps = 100 # Number of time steps
dt = T / num_steps # Time step size

# Create mesh
domain = mesh.create_unit_square(MPI.COMM_WORLD, 50, 50, cell_type=mesh.CellType.triangle)

# Create FunctionSpace
V = fem.functionspace(domain, ("Lagrange", 1))

# Define initial condition
def initial_condition(x, a=5):
    return np.exp(-a * (x[0]**2 + x[1]**2))

u_n = fem.Function(V)
u_n.name = "u_n"
u_n.interpolate(initial_condition)

# Apply Dirichlet BCs
fdim = domain.topology.dim - 1
# Identify the facets on the boundaries
facets = mesh.locate_entities_boundary(domain, fdim, lambda x: np.full(x.shape[1], True, dtype=bool))
# Locate DoFs on the facets
dofs_boundary = fem.locate_dofs_topological(V, fdim, facets)
# Create Dirichlet BCs
bc = fem.dirichletbc(PETSc.ScalarType(0.0), dofs_boundary, V)

# Time dependent output
results_folder = Path("results")
results_folder.mkdir(exist_ok=True, parents=True)
filename = results_folder / "out_tdht"
xdmf = io.XDMFFile(domain.comm, filename.with_suffix(".xdmf"), "w", io.XDMFFile.Encoding.ASCII)
xdmf.write_mesh(domain)
# Define solution variable, and interpolate initial solution for visualization in Paraview
uh = fem.Function(V)
uh.name = "uh"
uh.interpolate(initial_condition)
xdmf.write_function(uh, t)

# Define variational problem
# Define trial and test functions
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)
# Define source function
x = ufl.SpatialCoordinate(domain)
f = fem.Constant(domain, PETSc.ScalarType(0))
# Define variational form
a = u * v * ufl.dx + (dt/2) * ufl.dot(ufl.grad(u), ufl.grad(v)) * ufl.dx
L =  ((u_n + dt * f) * v - (dt/2) * ufl.dot(ufl.grad(u_n), ufl.grad(v))) * ufl.dx

# Assembly
bilinear_form = fem.form(a)
linear_form = fem.form(L)
A = assemble_matrix(bilinear_form, bcs=[bc])
A.assemble()
b = create_vector(linear_form)

# Solve variational problem
solver = PETSc.KSP().create(domain.comm)
solver.setOperators(A)
solver.setType(PETSc.KSP.Type.PREONLY)
solver.getPC().setType(PETSc.PC.Type.LU)

# Post-process
# Output visualisation
pyvista.start_xvfb()
grid = pyvista.UnstructuredGrid(*plot.vtk_mesh(V))
plotter = pyvista.Plotter()
plotter.open_gif("u_time.gif", fps=10)
grid.point_data["uh"] = uh.x.array
warped = grid.warp_by_scalar("uh", factor=1)
viridis = mpl.colormaps.get_cmap("viridis").resampled(25)
sargs = dict(title_font_size=25, label_font_size=20, fmt="%.2e", color="black",
             position_x=0.1, position_y=0.8, width=0.8, height=0.1)
renderer = plotter.add_mesh(warped, show_edges=True, lighting=False,
                            cmap=viridis, scalar_bar_args=sargs,
                            clim=[0, max(uh.x.array)])

# Time-stepping
for i in range(num_steps):
    t += dt
    # Update the right hand side reusing the initial vector
    with b.localForm() as loc_b:
        loc_b.set(0)
    assemble_vector(b, linear_form)
    # Apply Dirichlet boundary condition to the vector
    apply_lifting(b, [bilinear_form], [[bc]])
    b.ghostUpdate(addv=PETSc.InsertMode.ADD_VALUES, mode=PETSc.ScatterMode.REVERSE)
    set_bc(b, [bc])
    # Solve linear problem
    solver.solve(b, uh.x.petsc_vec)
    uh.x.scatter_forward()
    # Update solution at previous time step (u_n)
    u_n.x.array[:] = uh.x.array
    # Write solution to file
    xdmf.write_function(uh, t)
    # Update plot
    new_warped = grid.warp_by_scalar("uh", factor=1)
    warped.points[:, :] = new_warped.points
    warped.point_data["uh"][:] = uh.x.array
    plotter.write_frame()
plotter.close()
xdmf.close()